In [6]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [7]:
# TOGGLE ON/OFF RAW CODE
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [8]:
# IMPORTS

import os
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image, display, Math, Latex, HTML, Javascript
from ipywidgets import widgets, interact, Layout, interactive

%matplotlib inline
os.chdir('/home/fvanden/Documents/BAS/Python/aerosol-try/')

from mypysmps.io.read import read
from mypysmps.graph.psdisplay import PSDisplay
from mypysmps.smpswidgets.widget_aux import wchooseFile, wread

filename = widgets.Dropdown(options= wchooseFile(), value = 'data/testdata/LongTerm.csv', description='Filename')

In [11]:

display(filename)

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
button = widgets.Button(description="Apply")
button.on_click(run_all)

display(button)

<IPython.core.display.Javascript object>

Dropdown(description='Filename', index=8, options=('data/LongTerm.csv', 'data/LockdownData/2800720.csv', 'data…

Button(description='Apply', style=ButtonStyle())

In [12]:
SMPS = read(filename.value)
psd = PSDisplay(SMPS)

sample = widgets.IntSlider(min=0, max=len(SMPS.sample['data'])-1, continuous_update = False, description = 'sample #',layout = Layout(
            width = '100%'
        ))
#sfields = widgets.ToggleButtons(options=SMPS.data['variables'], value = 'normalised_number_concentration', description='Variable:', disabled=False, button_style='')
sfields = widgets.ToggleButtons(options=SMPS.data['variables'], value = SMPS.data['variables'][0], description='Variable:', disabled=False, button_style='')
D50line = widgets.Checkbox(value=False,description='D50',disabled=False,indent=False, layout = Layout(width='20%'))
boundaryline = widgets.Checkbox(value=False,description='Boundaries',disabled=False, indent=False, layout = Layout(width='20%'))
indperiods = widgets.Checkbox(value=False,description='24 hour periods',disabled=False,indent=False, layout = Layout(width='20%'))
clim = widgets.BoundedFloatText( value=2*10**4, min=1*10**1, max=1*10**8,step=1*10**1, description='clim:',
    style = {'description_width': 'initial'}, disabled=False, continuous_update=True, orientation='horizontal',
    readout=True, readout_format='.2f', layout = Layout(width='25%'))
ylim = widgets.BoundedFloatText( value=1*10**3, min=1*10**1, max=1*10**8,step=1*10**1, description='dlim:', disabled=False, continuous_update=True, orientation='horizontal',
    readout=True, readout_format='.2f', layout = Layout(width='25%'))

outplot =  widgets.interactive_output(psd.histogram,{'field':sfields,'sample':sample,'add_D50':D50line,'add_boundaries':boundaryline, 'xlim': ylim,'ylim':clim}) 
outplot2 = widgets.interactive_output(psd.plot, {'field':sfields,'clim':clim, 'ylim':ylim, 'indicator':sample,'periods':indperiods})
#outplot2 = widgets.interactive(psd.plot, field=sfields,clim=clim, ylim=ylim, indicator=sample)
date = widgets.interactive_output(psd.dateInfo,{'sample':sample})

ui1 = widgets.HBox([sfields])
ui2 = widgets.HBox([D50line, boundaryline, clim, ylim])
uidate = widgets.HBox([date])
ui3 = widgets.VBox([ui1, ui2, outplot, sample, uidate, outplot2, indperiods], layout = Layout(height='100%'))
                                      
display(ui3)